In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Load the training and testing data and store it in a variable
train_df = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
test_df = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')

In [ ]:
#looking through the training data
train_df.head()

In [ ]:
train_df.info()

In [ ]:
#looking into testing data
test_df.head()

In [ ]:
test_df.info()

In [ ]:
### cleaning the train data

#creating new columns and deleting duplicate columns

train_df = pd.concat([train_df, pd.get_dummies(train_df['DayOfWeek'])], axis=1)
train_df.drop('DayOfWeek', axis=1, inplace=True)
train_df = pd.concat([train_df, pd.get_dummies(train_df['PdDistrict'])], axis=1)
train_df.drop('PdDistrict', axis=1, inplace=True)
train_df.drop(['Address', 'Descript', 'Resolution', 'Dates'], axis=1, inplace=True)


In [ ]:
### cleaning the testing data

#creating new columns and deleting duplicate columns


test_df = pd.concat([test_df, pd.get_dummies(test_df['DayOfWeek'])], axis=1)
test_df.drop('DayOfWeek', axis=1, inplace=True)
test_df = pd.concat([test_df, pd.get_dummies(test_df['PdDistrict'])], axis=1)
test_df.drop('PdDistrict', axis=1, inplace=True)
test_df.drop(['Address', 'Dates'], axis=1, inplace=True)

In [ ]:
# looking into data after proccessing
train_df.info()

In [ ]:
test_df.info()

In [ ]:
# Preparing the data for training and testing
x_train=(train_df.drop(['Category'],axis=1))
y_train=train_df['Category']

In [ ]:
# Creating dummies for y train
y_train=pd.get_dummies(y_train)
y_train.head()

In [ ]:
# Splitting test and train data

from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x_train,y_train,test_size=0.2)

In [ ]:
# Import the libraries for the network

from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout

In [ ]:
# Create a sequencial network

model = Sequential()
model.add(Dense(256, input_shape=(x_train.shape[1], )))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(39, activation='softmax'))

In [ ]:
# Compile the model with appropriate optimizer

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model

model.fit(xtrain,ytrain,
         batch_size=256,
         epochs=10,
         verbose=2,
         validation_data=(xtest,ytest))

In [ ]:
# Predict the outcome

prediction=model.predict(test_df.drop('Id',axis=1))

In [ ]:
sample_submission = pd.read_csv('../input/sf-crime/sampleSubmission.csv.zip')

In [ ]:
# Prepare the final submission file

prediction1=[np.argmax(i) for i in prediction]
predictionshape = np.max(prediction, axis=1).reshape(-1, 1)
prediction1 = np.array((prediction == predictionshape), dtype='int32')
column=list(sample_submission.columns)
column.remove('Id')
submission1=pd.DataFrame({'Id':test_df['Id']})
for i,v in enumerate(column):
    submission1[v]=prediction1[:,i]


In [ ]:
submission1.head()

In [ ]:
# Conbvert the file to csv file

submission1.to_csv('../working/submission1.csv', index=False)